In [3]:
import tensorflow_datasets as tfds


ds, info = tfds.load("reddit", split="train", with_info=True, data_dir="~/shared/ml/tensorflow")

# Take the first five rows
first_five_rows = ds.take(5)

# Iterate over the first five rows and print them
for row in first_five_rows:
    print(row)

{'author': <tf.Tensor: shape=(), dtype=string, numpy=b'huh7'>, 'body': <tf.Tensor: shape=(), dtype=string, numpy=b"I  was talking to this girl I've met today, and I'm not so sure what happened. It was nice seeing her after almost 3 weeks last talking with her, and she seems like a cool person to hang out with. I realized however, that I want to try to be good friends with her. We walked around and talked a while, and before I knew what happened I asked and she gave me her digits. \n\nWe proceeded to text each other back and forth about lots of bullshit, and I suddenly asked myself; is this emotionally cheating? To be honest, I do feel as if I did. \n\nI understand I (19M)am in a monogamous relationship with my gf(17), and I want communication between us to be crystal clear. I feel guilty, and an obligation to tell her of what I did. Should I tell my gf about this, or stop myself in the future before something like this happens again? I am emotionally confused right now, and could use s

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

conf = SparkConf().set("spark.jars.packages", "com.linkedin.sparktfrecord:spark-tfrecord_2.12:0.6.0")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)
print(f'The PySpark {spark.version} version is running...')

:: loading settings :: url = jar:file:/opt/conda/envs/bigdata/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.linkedin.sparktfrecord#spark-tfrecord_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d1bfc61d-a0d7-45fe-934c-5672fe959641;1.0
	confs: [default]
	found com.linkedin.sparktfrecord#spark-tfrecord_2.12;0.6.0 in central
:: resolution report :: resolve 313ms :: artifacts dl 13ms
	:: modules in use:
	com.linkedin.sparktfrecord#spark-tfrecord_2.12;0.6.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spar

The PySpark 3.5.0 version is running...


In [3]:
# Directory containing TFRecord files and the JSON file
tfrecord_dir = "shared/ml/tensorflow/reddit/1.0.0/*"

reddit_data = spark.read.format("tfrecord") \
    .option("recordType", "Example") \
    .option("pathGlobFilter", "*.tfrecord-*") \
    .load(tfrecord_dir)


reddit_data.show(3)

+--------------------+------------+---------------+--------------------+--------------------+--------------------+------------+-------+
|      normalizedBody|subreddit_id|      subreddit|             summary|                body|             content|      author|     id|
+--------------------+------------+---------------+--------------------+--------------------+--------------------+------------+-------+
|I did. It was one...|    t5_2qh1i|      AskReddit|I cheated with a ...|I did. It was one...|I did. It was one...|RandomGuy0-2|ceb578v|
|My son was 3 when...|    t5_2qh1i|      AskReddit|kids who cannot s...|My son was 3 when...|My son was 3 when...|      Logun0|cart4tc|
|Okay, so I take s...|    t5_2tkk8|OpiatesRecovery|Opiates give a po...|Okay, so I take s...|Okay, so I take s...|  seestraw89|c7hsue4|
+--------------------+------------+---------------+--------------------+--------------------+--------------------+------------+-------+
only showing top 3 rows



In [4]:
reddit_data.count()

3848330

In [6]:
from pyspark.sql.functions import *

# Getting number of texts per subreddit
summary_per_subreddit = reddit_data.groupby("subreddit").count().orderBy(col('count').desc())
summary_per_subreddit.show()

+-------------------+------+
|          subreddit| count|
+-------------------+------+
|          AskReddit|589947|
|      relationships|352049|
|    leagueoflegends|109307|
|               tifu| 52219|
|relationship_advice| 50416|
|              trees| 47286|
|             gaming| 43851|
|            atheism| 43268|
|      AdviceAnimals| 40783|
|              funny| 40171|
|           politics| 36518|
|               pics| 35098|
|                sex| 28806|
|                WTF| 25781|
|  explainlikeimfive| 25482|
|      todayilearned| 25004|
|            Fitness| 22694|
|               IAmA| 22689|
|          worldnews| 22577|
|              DotA2| 22405|
+-------------------+------+
only showing top 20 rows

